# Computer vision and deep learning - Laboratory 4

This laboratory will span over two sessions, and you will exercise more of your torch skills. Also, you will learn how to perform experiment tracking using the popular wandb framework.

We'll be implementing, training and testing a semantic segmentation model. The task of semantic segmentation implies the labeling/classification of __all__ the pixels in the input image.

You'll build and train a fully convolutional neural network inspired by U-Net.
Also, you will learn about how you can use various callbacks during the training of your model.


Today, we'll focus on the data loading and preprocessing part, and we'll study the building blocks of the semantic segmentation module.

In [1]:

import torch.nn.functional

from model import UNet
from lfw_dataset import LFWDataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
import cv2
import matplotlib.pyplot as plt
from utils import transform_generator, inv_transform
from copy import deepcopy
import pickle
from utils import eval
from train import train
import wandb

In [ ]:
sweep_config = {
    'method': 'grid'
    }
metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'NUM_EPOCHS': {
        'values': [10]
    },
     'BATCH_SIZE': {
         'values': [32, 48, 64, 80, 96]
      },
    'INPUT_SHAPE': {
        'values': [(64, 64)]
    },
    'NUM_LAYERS': {
        'values': [2]
    },
    'LR': {
        'values': [0.004, 0.008, 0.01]
      },
    'INTERMEDIARY_FILTERS': {
        'values': [8]
    }
}

# parameters_dict = {
#     'NUM_EPOCHS': {
#         'values': [10, 20]
#     },
#      'BATCH_SIZE': {
#         # integers between 32 and 256
#         # with evenly-distributed logarithms 
#         'distribution': 'q_log_uniform_values',
#         'q': 8,
#         'min': 32,
#         'max': 96,
#       },
#     'INPUT_SHAPE': {
#         'values': [(64, 64)]
#     },
#     'NUM_LAYERS': {
#         'values': [1, 2]
#     },
#     'LR': {
#         # a flat distribution between 0 and 0.1
#         'distribution': 'uniform',
#         'min': 0.004,
#         'max': 0.04
#       },
#     'INTERMEDIARY_FILTERS': {
#         'values': [8, 16, 32]
#     }
# }

sweep_config['parameters'] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="cvdl")

In [ ]:
def run_wandb():
    with wandb.init():
        train(None, wandb.config)

In [ ]:
wandb.agent(sweep_id, run_wandb, project="cvdl")

In [2]:
wandb.init(
    # set the wandb project where this run will be logged
    project="cvdl",
    
    # track hyperparameters and run metadata
    config={
        'NUM_EPOCHS': 30,
        'BATCH_SIZE': 32,
        'INPUT_SHAPE': (128, 128),
        'NUM_LAYERS': 2,
        'LR': 0.004,
        'INTERMEDIARY_FILTERS': 32,
        'OPTIMIZER': 'Adam'
    }
)

wandb: Currently logged in as: moldovandangelu. Use `wandb login --relogin` to force relogin


In [3]:
wandb.config

{'NUM_EPOCHS': 30, 'BATCH_SIZE': 32, 'INPUT_SHAPE': [128, 128], 'NUM_LAYERS': 2, 'LR': 0.004, 'INTERMEDIARY_FILTERS': 32, 'OPTIMIZER': 'Adam'}

In [ ]:
NUM_EPOCHS = 100
BATCH_SIZE= 128
INPUT_SHAPE = (64, 64)
NUM_LAYERS = 2
LR = 0.01

ARTIFACTS_PATH='./artifacts'
BASE_PATH="./lfw_dataset"

In [ ]:
model = train(None, wandb.config)[0]

Epoch [1/30], Train Loss: 0.6123, Validation Loss: 0.7531


In [ ]:
wandb.finish()